In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization


import numpy as np
from glob import glob

from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path ='/kaggle/input/hepeople-ex/Train_New'
valid_path ='/kaggle/input/find_label1/Test'

In [ ]:
# # TOp is input layer so we define size and 3 is for the channels
# # Here we will be using imagenet weights

#inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False,pooling='max')

In [ ]:
# TOp is input layer so we define size and 3 is for the channels
# Here we will be using imagenet weights
import tensorflow
inception =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False,pooling='max')

In [ ]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False
    
# useful for getting number of output classes
folders = glob('/kaggle/input/hepeople-ex/Train_New')

In [ ]:
# model = Sequential([
#   inception, 
#   Dense(1024, activation='relu'),
#   Dropout(0.25),
#   Dense(256, activation='relu'),
#   Dropout(0.25),
#   Dense(3, activation='softmax')
# ])

In [ ]:
# # our layers - you can add more if you want
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(3, activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
#                                          factor=0.1,
#                                          patience=2,
#                                          cooldown=2,
#                                          min_lr=0.00001,
#                                          verbose=1)

# callbacks = [reduce_learning_rate]

In [ ]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/kaggle/input/hepeople-ex/Train_New',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/kaggle/input/find-label1/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  #callbacks=callbacks,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
model.save('model_inception_V3.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
fnames = [str(x)[str(x).rfind('/')+1:] for x in test_set.filenames]

In [ ]:
l = dict((v,k) for k,v in training_set.class_indices.items())
prednames = [l[k] for k in y_pred]

In [ ]:
df_final = pd.DataFrame({'Filename': fnames,'Category': prednames})
df_final.head(3)

In [ ]:
df_final.to_csv('submission.csv', index=False)